# Building an Image Classifier with Pytorch

In [1]:
# import packages and practice dataset
import torch 
from torch import nn
from torch import optim
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader

In [2]:
# Load and split the dataset 
train_data = datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor())
train, val = random_split(train_data, [55000, 5000])
# Push data to DataLoader
train_loader = DataLoader(train, batch_size=32)
val_loader = DataLoader(val, batch_size=32)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw




Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw




Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
# Define the model
# model = nn.Sequential(
#     nn.Linear(28 * 28, 64),
#     nn.ReLU(),
#     nn.Linear(64, 64),
#     nn.ReLU(),
#     nn.Dropout(0.1),
#     nn.Linear(64, 10)
# )

In [4]:
# Define model class with residual connections using nn.Module

class ResNet(nn.Module):
  # define the class attribues
  def __init__(self):
      super().__init__()
      # input layer (input image dimensions and hidden dimension)
      self.l1 = nn.Linear(28*28, 64)
      # second hidden layer
      self.l2 = nn.Linear(64, 64)
      # output layer hidden dim and output dim
      self.l3 = nn.Linear(64, 10)
      # dropout layer
      self.do = nn.Dropout(0.1)

 
  def forward(self, x):
    # Push the data to the first layer
    h1 = nn.functional.relu(self.l1(x))
    # Push tghe output of the first layer through the second
    h2 = nn.functional.relu(h1)
    # Residual connection allows the optimizer to reduce or disuse l1
    do = self.do(h2 + h1)
    # Final dropout layer to prevent overfitting
    logits = self.l3(do)
    # Return the logits
    return logits

# Push the model the GPU
model = ResNet().cuda()

In [5]:
# Define optimizer with standard model paramneters and learning rate
optimizer = optim.SGD(model.parameters(), lr=1e-2)

In [6]:
# Define loss function
loss = nn.CrossEntropyLoss()

In [7]:
# train loop
n_epochs = 5
# Set the model to training
model.train()
for epoch in range(n_epochs):
  # lists to capture the outputs
  losses = []
  accuracies = []
  # Loop through the dataloader batches 
  for batch in train_loader:
    # pull the image and labels from the batch
    x, y = batch
    # reshape the input from image 28*28 to a 1D vector and push x to GPU
    b = x.size(0)
    x = x.view(b, -1).cuda()

    # 1 Compute the logit 
    logit = model(x) 
    # 2 compute the loss
    l = loss(logit, y.cuda())
    # 3 clean the gradients
    model.zero_grad()
    # 4 accumulate the partial derivatives 
    l.backward()
    # 5 step in the opposite direction of the gradient
    optimizer.step()
    # with torch.no_grad():
       # params = (previous) params - eta(lr) * params.grad (gradeints)

    losses.append(l.item())
    accuracies.append(y.eq(logit.detach().argmax(dim=-1).cpu()).float().mean())

  print(f"Epoch {epoch + 1}")
  print(f"train_loss {torch.tensor(losses).mean():.2f}",f"train acc {torch.tensor(accuracies).mean():.2f}")

  # val loop
  model.eval()
  for batch in val_loader:
    losses = []
    accuracies = []
    x, y = batch

    b = x.size(0)
    x = x.view(b, -1).cuda()
    # ignore the gradients we dont need to train on the validation
    with torch.no_grad():
      logit = model(x) 

    l = loss(logit, y.cuda())

    losses.append(l.item())
    accuracies.append(y.eq(logit.detach().argmax(dim=-1).cpu()).float().mean())

  print(f"val_loss {torch.tensor(losses).mean():.2f}",f"val acc {torch.tensor(accuracies).mean():.2f}\n")




Epoch 1
train_loss 0.65 train acc 0.83
val_loss 0.15 val acc 1.00

Epoch 2
train_loss 0.31 train acc 0.91
val_loss 0.09 val acc 1.00

Epoch 3
train_loss 0.26 train acc 0.93
val_loss 0.06 val acc 1.00

Epoch 4
train_loss 0.23 train acc 0.94
val_loss 0.04 val acc 1.00

Epoch 5
train_loss 0.20 train acc 0.94
val_loss 0.03 val acc 1.00

